In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from tqdm.contrib import tzip, tenumerate, tmap

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
import pyspark.sql.types as pstype
import pyspark.sql.functions as F
import pyspark as ps

import matplotlib as mlt
import matplotlib.pyplot as plt
import japanize_matplotlib

from time_series_model import *

%matplotlib inline
%matplotlib ipympl

In [2]:
np.set_printoptions(threshold=100000, precision=4, linewidth=10000)
ps_conf = ps.SparkConf().set("spark.logConf", "false")\
            .set("spark.executor.memory", "12g")\
            .set("spark.driver.memory", "4g")\
            .set("spark.executor.cores", "7")\
            .set("spark.sql.shuffle.partitions", "500")\
            .set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+UseStringDeduplication")\
            .set("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1 Young Generation")\
            .set("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1 Old Generation")\
			.set("spark.logConf", "false")
spark = SparkSession.builder.config(conf=ps_conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 13:10:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
SPECIFIED_PATH = "csv_data/"
SPECIFIED_DATE = "20241120"
SPECIFIED_CSV  = SPECIFIED_PATH + SPECIFIED_DATE

In [4]:
input_path  = SPECIFIED_CSV + "_raw_data_sumida_bunka.csv"
df_raw_data = spark.read.option("inferSchema", "True").option("header", "True").csv(input_path)
df_raw_data.persist(StorageLevel.MEMORY_AND_DISK_DESER)

utid_list = sorted(df_raw_data.select("unit_id").drop_duplicates().rdd.flatMap(lambda x: x).collect())

24/11/28 13:10:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df_raw_data\
		.filter(col("unit_id").isin(utid_list))\
		.filter(col("randomized") == 1)\
		.select(['date', 'datetime', 'unit_id', 'aibeaconid'])\
        .withColumn('minute', F.date_trunc('minute', 'datetime'))\
        .withColumn('minute', F.unix_timestamp('minute'))\
        .withColumn('minute', col('minute') - (col('minute') % 60))\
        .withColumn('minute', F.from_unixtime('minute'))\
        .select(['date', 'minute', 'unit_id', 'aibeaconid'])\
        .drop_duplicates()\
        .groupBy(['date', 'minute', 'unit_id'])\
        .agg(F.count('*').alias('1min_count'))\
        .orderBy(['date', 'minute', 'unit_id'])\
        .toPandas()\
        .to_csv(SPECIFIED_CSV + "_1min_data_sumida_bunka.csv", header=True, index=False)

24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/11/28 13:11:06 WARN RowBasedKeyValueBatch: Calling spill() on

In [6]:
SPECIFIED_PAST = (datetime.datetime.strptime(SPECIFIED_DATE, '%Y%m%d') - relativedelta(days=1) + datetime.timedelta(hours=9)).strftime('%Y-%m-%d')

start_time = f'{SPECIFIED_PAST} 00:00:00'
end_time = f'{SPECIFIED_PAST} 23:59:00'
# 1分単位の時間列を作成
time_range = pd.date_range(start=start_time, end=end_time, freq='min')
# DataFrameに変換
df_time = pd.DataFrame(time_range, columns=['datetime'])
df_time = spark.createDataFrame(df_time)\
				.withColumn('hour', F.hour(col('datetime')))

df_by1min = spark.read\
				.option('header', True)\
				.option('inferSchema', True)\
           		.csv(SPECIFIED_CSV + "_1min_data_sumida_bunka.csv")

for unit_id in utid_list:
    df_tmp  = df_by1min\
        		.filter(col('unit_id') == unit_id)\
          		.select(['minute', '1min_count'])\
            	.withColumnRenamed('minute',     'datetime')\
                .withColumnRenamed('1min_count', unit_id)
    df_time = df_time.join(df_tmp, on='datetime', how='left')
df_time = df_time\
			.fillna(0)\
    		.orderBy('datetime')
df_time.persist(StorageLevel.MEMORY_AND_DISK_DESER)
df_time.show()

24/11/28 13:11:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|           datetime|hour|1002A25B|1002A25C|1002A25D|1002A5BA|1002A5BB|1002A5BD|1002A5BE|1002A5BF|1002A5C1|1002A5C3|1002A5C4|1002A5C5|1002A5C6|1002A5C7|1002A5C9|1002A5CA|1002A5CB|1002A5CC|1002A5CE|1002A5CF|1002A5D0|1002A5D1|1002A6A5|1002A6A7|1002A6A8|1002A6AF|1002A6B0|1002A6B2|1002A6B3|1002A6B6|1002A6B9|1002A6BA|1002A6BC|1002A6BD|1002A6BE|10

In [7]:
SPECIFIED_HOUR = 14

pd_data = df_time.toPandas()
pd_data = pd_data[pd_data['hour'] == SPECIFIED_HOUR]
pd_data = pd_data[utid_list]
pd_data

,1002A25B,1002A25C,1002A25D,1002A5BA,1002A5BB,1002A5BD,1002A5BE,1002A5BF,1002A5C1,1002A5C3,...,1002ABE5,1002ABE8,1002ABF2,1002ABF3,1002ACA9,1002ACAA,1002ACB0,1002ACB2,1002ACB8,1002ACB9
840,27,29,41,31,30,36,1,18,24,0,...,41,49,5,42,57,21,67,2,11,8
841,19,18,53,25,23,36,3,18,24,3,...,45,52,5,32,49,17,64,6,20,2
842,10,14,52,18,15,13,1,14,12,0,...,56,67,14,48,65,10,71,8,15,12
843,7,23,72,27,13,20,3,15,8,0,...,52,43,3,45,56,18,66,0,21,0
844,6,24,65,27,15,27,2,11,10,0,...,61,72,5,51,80,14,81,3,28,1
845,5,16,57,22,15,22,2,14,5,0,...,48,59,5,43,68,10,73,2,13,2
846,9,20,42,20,16,21,2,14,16,0,...,67,91,7,68,97,14,84,7,15,5
847,11,18,41,24,19,19,5,14,18,0,...,71,105,6,71,101,17,87,7,13,5
848,5,17,64,23,11,19,3,7,10,0,...,73,84,6,64,85,13,89,5,16,3
849,11,24,48,30,13,21,4,13,10,2,...,79,65,5,61,75,8,96,5,18,2


In [8]:
test_data = pd_data.values.tolist()
test_data

[[27,
  29,
  41,
  31,
  30,
  36,
  1,
  18,
  24,
  0,
  37,
  59,
  10,
  12,
  21,
  4,
  59,
  55,
  31,
  52,
  50,
  30,
  51,
  16,
  4,
  44,
  12,
  39,
  51,
  46,
  10,
  3,
  37,
  3,
  33,
  26,
  40,
  47,
  0,
  34,
  42,
  40,
  56,
  38,
  18,
  16,
  49,
  66,
  76,
  24,
  5,
  46,
  0,
  41,
  7,
  55,
  76,
  33,
  22,
  40,
  41,
  49,
  5,
  42,
  57,
  21,
  67,
  2,
  11,
  8],
 [19,
  18,
  53,
  25,
  23,
  36,
  3,
  18,
  24,
  3,
  28,
  52,
  22,
  15,
  20,
  6,
  62,
  58,
  22,
  34,
  40,
  27,
  45,
  24,
  6,
  42,
  15,
  45,
  47,
  42,
  11,
  6,
  33,
  3,
  33,
  35,
  23,
  59,
  0,
  36,
  39,
  25,
  75,
  41,
  23,
  22,
  55,
  69,
  65,
  17,
  9,
  51,
  0,
  44,
  7,
  45,
  71,
  14,
  12,
  36,
  45,
  52,
  5,
  32,
  49,
  17,
  64,
  6,
  20,
  2],
 [10,
  14,
  52,
  18,
  15,
  13,
  1,
  14,
  12,
  0,
  22,
  59,
  16,
  5,
  14,
  15,
  75,
  53,
  25,
  34,
  48,
  12,
  70,
  22,
  16,
  59,
  24,
  55,
  54,
  44,
  12,
 

In [9]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="external library", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

平均二乗誤差(MSE): 43.133819179668926
L2正則化項(l2 norm): 1.2440372103772035
L1正則化項(l1 norm): 35.61168441837656
目的関数(Objective):  1669.2972779195584
目的関数(Objective)の微分:  82286.31137999459


np.float64(6976.363176383552)

In [10]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="coordinate descent", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean = model.predict(x_tmp)

# print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

ite:1  mse:0.04293531461587618  ΔDiff:1.5506055650903185
ite:2  mse:0.015241638191323058  ΔDiff:0.9238678145244
ite:3  mse:0.002195931228539626  ΔDiff:0.35067384960703735
ite:4  mse:0.0006546441328189439  ΔDiff:0.19146819954723776
ite:5  mse:0.00025367068948007655  ΔDiff:0.11918707400924097
ite:6  mse:0.00011552935292788545  ΔDiff:0.08043409578009555
ite:7  mse:5.774426849092892e-05  ΔDiff:0.05686544676244107
ite:8  mse:3.119032871518735e-05  ΔDiff:0.04179304257948315
ite:9  mse:1.7398183130812315e-05  ΔDiff:0.031213751061439084
ite:10  mse:9.540644645551155e-06  ΔDiff:0.023114413255604493
ite:11  mse:5.298071214456817e-06  ΔDiff:0.017224749287277935
ite:12  mse:3.1164359695605874e-06  ΔDiff:0.013210617483501401
ite:13  mse:1.9691436277128696e-06  ΔDiff:0.010501049621438835
ite:14  mse:1.2499208449639772e-06  ΔDiff:0.008366335357728778
ite:15  mse:7.468675224036049e-07  ΔDiff:0.006467192687294996
ite:16  mse:4.324982041647614e-07  ΔDiff:0.004921371702404385
ite:17  mse:2.55019692698631

np.float64(6976.363176361882)

In [11]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=30000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="ISTA", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

ite:1  mse:193787.5789572286  update_diff:1853.0725013600877 diff:193787.5789572286
ite:2  mse:17234.32824396892  update_diff:81.40879946865039 diff:176553.25071325968
ite:3  mse:7705.06113932743  update_diff:19.057710439929856 diff:9529.26710464149
ite:4  mse:4808.997361495405  update_diff:10.122710255839854 diff:2896.063777832025
ite:5  mse:3192.5311552486164  update_diff:5.91615546943392 diff:1616.4662062467887
ite:6  mse:2220.539727380031  update_diff:3.5873624977165353 diff:971.9914278685856
ite:7  mse:1612.042559614722  update_diff:2.2584374826898985 diff:608.4971677653089
ite:8  mse:1215.0483755898347  update_diff:1.4786305640728035 diff:396.9941840248873
ite:9  mse:945.2726573062045  update_diff:1.006706681952748 diff:269.7757182836302
ite:10  mse:754.5167807817124  update_diff:0.7108489888174184 diff:190.755876524492
ite:11  mse:614.8101412325951  update_diff:0.5185259949056398 diff:139.7066395491173
ite:12  mse:509.36571424505973  update_diff:0.3889894367018215 diff:105.44442

np.float64(6976.363177286648)

In [12]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=30000000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="FISTA", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

# print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

ite:1  mse:198233.44527322132  update_diff:1896.2042192696156 diff:198233.44527322132
ite:2  mse:198233.44527322132  update_diff:1896.2042192696156 diff:0.0
ite:3  mse:17566.035397709515  update_diff:83.32605840947512 diff:180667.4098755118
ite:4  mse:6556.645889102351  update_diff:15.082999072365856 diff:11009.389508607164
ite:5  mse:3126.617477269815  update_diff:5.977403227514577 diff:3430.028411832536
ite:6  mse:1462.2648317032774  update_diff:1.794850733892368 diff:1664.3526455665376
ite:7  mse:777.1783929491442  update_diff:0.6192556805488852 diff:685.0864387541333
ite:8  mse:478.4231150851144  update_diff:0.3493222879935628 diff:298.7552778640298
ite:9  mse:312.32058687591524  update_diff:0.24917174998085836 diff:166.10252820919914
ite:10  mse:201.12578787910863  update_diff:0.15604318037925252 diff:111.19479899680661
ite:11  mse:128.18257969718246  update_diff:0.08316964654794233 diff:72.94320818192617
ite:12  mse:85.11558872978205  update_diff:0.046032053097773126 diff:43.0669

np.float64(6976.363207693468)

In [13]:
SPECIFIED_PATH = "csv_data/"
SPECIFIED_DATE = "2024-11-13"
SPECIFIED_CSV  = SPECIFIED_PATH + SPECIFIED_DATE

pd_test_data = spark.read\
				.option('header', True)\
				.option('inferSchema', True)\
           		.csv(SPECIFIED_CSV + "_time_series_data.csv")\
				.toPandas()

In [14]:
mean_sorted_list = pd_test_data.mean().sort_values(ascending=False).index
pd_test_data     = pd_test_data[mean_sorted_list].fillna(0).reset_index(drop=True)
pd_test_data     = pd_test_data.values.tolist()

In [15]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="external library", visible_flg=False)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


np.float64(10002.2745763755)

In [16]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=1000000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="coordinate descent", visible_flg=False)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

np.float64(10002.274576429503)

In [17]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=1000000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="ISTA", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

ite:1  mse:831767.7153550242  update_diff:4905.971749760169 diff:831767.7153550242
ite:2  mse:1369.0902322002044  update_diff:2.2152108025832664 diff:830398.625122824
ite:3  mse:917.835813436055  update_diff:0.26575940096464806 diff:451.25441876414936
ite:4  mse:812.9340993518637  update_diff:0.1974502341261653 diff:104.90171408419133
ite:5  mse:731.1222217588974  update_diff:0.15062851368261956 diff:81.81187759296631
ite:6  mse:665.4713468230419  update_diff:0.11793274572700398 diff:65.65087493585543
ite:7  mse:611.3432183180971  update_diff:0.09474692218628268 diff:54.12812850494481
ite:8  mse:565.6503492001297  update_diff:0.07802599823103569 diff:45.69286911796746
ite:9  mse:526.227814616917  update_diff:0.06570511187362445 diff:39.42253458321261
ite:10  mse:491.61058101786494  update_diff:0.056419141561925 diff:34.6172335990521
ite:11  mse:460.74991080270746  update_diff:0.049241472840332465 diff:30.860670215157484
ite:12  mse:432.9194703745784  update_diff:0.04354924923478231 dif

np.float64(10002.274581097801)

In [18]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=1000000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="FISTA", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
mse

ite:1  mse:813042.5943947602  update_diff:4795.561249392957 diff:813042.5943947602
ite:2  mse:813042.5943947602  update_diff:4795.561249392957 diff:0.0
ite:3  mse:1342.3345181715465  update_diff:2.115907886076975 diff:811700.2598765887
ite:4  mse:905.19915523976  update_diff:0.3330899164647833 diff:437.1353629317865
ite:5  mse:750.9571709029137  update_diff:0.13764090495714426 diff:154.24198433684626
ite:6  mse:638.1926037503648  update_diff:0.0893167895256082 diff:112.76456715254892
ite:7  mse:543.5512541325812  update_diff:0.060738524476102354 diff:94.64134961778359
ite:8  mse:462.51959638725003  update_diff:0.04418883706017672 diff:81.03165774533119
ite:9  mse:391.3469377789776  update_diff:0.033962365625658505 diff:71.17265860827246
ite:10  mse:327.923536243477  update_diff:0.02652985695555904 diff:63.4234015355006
ite:11  mse:271.44608051746945  update_diff:0.020337106554504915 diff:56.47745572600752
ite:12  mse:221.9024445212259  update_diff:0.015021262071189912 diff:49.543635996

np.float64(10002.274592834197)